In [20]:
import data1
import torch, torch.utils, torch.utils.data
from unet import UNet
from tqdm.notebook import tqdm as tqdm
from torchsummary import summary
import matplotlib.pyplot as plt
import modeltools1
import datetime
import random
import torchvision
import os
import filters1
from itertools import product

device = torch.device("cuda")
weights_path = "weights-FocalLoss-30000epochs.dat"
out_dir = "out1"
if not os.path.exists(out_dir): os.mkdir(out_dir)
#weights_path = None

In [4]:
train_data_dir = "../01_image_segmentation1/01_train"
train_data_filename = "idx-train.txt" #should change to _orig?
dataset_train = data1.Dataset(train_data_dir, train_data_filename)

val_data_dir = "../01_image_segmentation1/02_test_clean"
val_data_filename = "idx-test.txt"
dataset_val = data1.Dataset(val_data_dir, val_data_filename)
dataset_val.eval()

dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 4, num_workers=1, 
    pin_memory=True, persistent_workers=True)

dataloader_val = torch.utils.data.DataLoader(dataset_train, batch_size = 4, num_workers=1, 
    pin_memory=True, persistent_workers=True)

Loading raw data: 0it [00:00, ?it/s]

Loading raw data: 0it [00:00, ?it/s]

In [5]:
model = UNet(in_channels = 3, out_channels=3, n_blocks = 4).to(device)
#uncomment to check the model summary
summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
              ReLU-2         [-1, 32, 128, 128]               0
       BatchNorm2d-3         [-1, 32, 128, 128]              64
            Conv2d-4         [-1, 32, 128, 128]           9,248
              ReLU-5         [-1, 32, 128, 128]               0
       BatchNorm2d-6         [-1, 32, 128, 128]              64
         MaxPool2d-7           [-1, 32, 64, 64]               0
         DownBlock-8  [[-1, 32, 64, 64], [-1, 32, 128, 128]]               0
            Conv2d-9           [-1, 64, 64, 64]          18,496
             ReLU-10           [-1, 64, 64, 64]               0
      BatchNorm2d-11           [-1, 64, 64, 64]             128
           Conv2d-12           [-1, 64, 64, 64]          36,928
             ReLU-13           [-1, 64, 64, 64]               0
      BatchNorm2d-14      

In [6]:
lr = 0.01
optim_params = model.parameters()
optim = torch.optim.Adam(optim_params, lr=lr)
# loss = torch.nn.CrossEntropyLoss()
loss = modeltools1.FocalLoss(reduction="mean")
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma =0.9999)#gamma=0.997)

In [7]:
losses = []
if ("weights_path" in locals() or "weights_path" in globals()) and weights_path:
    model.load_state_dict(torch.load(weights_path))
else:
    modeltools1.train_model(model, loss, optim, scheduler, dataloader_train,
        num_epochs = 1000, device = device) #30 000 takes one hour and suffices
    model.cpu()
    current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    torch.save(model.state_dict(), './weights'+current_time+".dat")

In [8]:
# dataset_train.eval()
# image = dataset_train[0][0]
# label = dataset_train[0][1]
# label_masks = modeltools1.classes_to_masks(label)


# model.cuda()

# prediction = modeltools1.tiled_eval(model, image, 16, (128,128), (64,64), device, device)

In [9]:
# pred_mask = modeltools1.logits_to_masks(prediction)
# masked = modeltools1.apply_mask(image, pred_mask, colors=modeltools1.DEFAULT_OVERLAY_COLORS, alpha=0.3)
# torchvision.io.write_png(masked, out_dir+"/test.png")
# score = modeltools1.F1_score(pred_mask, label_masks)

In [10]:

dataset_train.eval()
total_score = modeltools1.eval_on_dataset(model, dataset_train, out_dir+"/train", 8,
    (128,128), (64,64), device, device, [])

print(f"Average F1 score on train set: {total_score}")

Images in dataset:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

Score:  0.8979603275315912


  0%|          | 0/51 [00:00<?, ?it/s]

Score:  0.9183278604105028


  0%|          | 0/30 [00:00<?, ?it/s]

Score:  0.9096528753406101


  0%|          | 0/102 [00:00<?, ?it/s]

Score:  0.9245349943980967
Average F1 score on train set: 0.9126190144202002


In [19]:
dataset_train.eval()
image, label = dataset_train[3]
prediction = modeltools1.tiled_eval(model, image, 16, (128,128), (64,64), device, device)
pred_mask = modeltools1.logits_to_masks(prediction)
label_mask = modeltools1.classes_to_masks(label)
score = modeltools1.F1_score(pred_mask, label_mask)

pred_fitered = filters1.remove_small_holes(pred_mask, 20, 50)
filtered_score = modeltools1.F1_score(pred_fitered, label_mask)

print(f"Original score: {score}, filtered score: {filtered_score}")

  0%|          | 0/51 [00:00<?, ?it/s]

Original score: 0.9245349943980967, filtered score: 0.9426969245358731, double_filetered 0.9480136641971647


In [22]:
#Parameter search on train (honesty!) set
closure_rads = range(5, 100, 5)
areas = range(5, 100, 5)

initial_masks = []
labels_masks = []
#Generate original masks
bestscore = 0
bestparams = None
for i in tqdm(range(len(dataset_train)), "Initial masks"):
    image, label = dataset_train[i]
    prediction = modeltools1.tiled_eval(model, image, 16, (128,128), (64,64), device, device)
    pred_mask = modeltools1.logits_to_masks(prediction)
    label_mask = modeltools1.classes_to_masks(label)
    score = modeltools1.F1_score(pred_mask, label_mask)
    initial_masks.append(pred_mask)
    labels_masks.append(label_mask)
    bestscore += score
bestscore /= len(dataloader_train) 
print("Initial score: ", bestscore)

#Randomize search grid
params =[*product(closure_rads, areas)]
random.shuffle(params)

#do search
for clrad, area in tqdm(params, "Parameters search"):
    score = 0
    for i in range(len(dataset_train)):
        pred_fitered = filters1.remove_small_holes(initial_masks[i], clrad, area)
        score += modeltools1.F1_score(pred_fitered, labels_masks[i])
    score /= len(dataset_train)
    if (score > bestscore):
        bestscore = score
        bestparams = clrad, area
        print(f"Best score: {score} with closure rad {clrad} and area {area}")    
    break


Initial masks:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

Initial score:  3.698139977592387


Parameters search:   0%|          | 0/361 [00:00<?, ?it/s]

In [11]:
# total_score = modeltools1.eval_on_dataset(model, dataset_val, out_dir+"/val", 8,
#     (128,128), (64,64), device, device, [])

# print(f"Average F1 score on val set: {total_score}")

Images in dataset:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

Score:  0.8628483556146254


  0%|          | 0/41 [00:00<?, ?it/s]

Score:  0.9321581267261329


  0%|          | 0/58 [00:00<?, ?it/s]

ValueError: Found input variables with inconsistent numbers of samples: [1979450, 1974875]

In [ ]:
total_score

0.9030639571772393